In [1]:
import torch
import numpy as np
from torch import optim
import os
import matplotlib.pyplot as plt

In [2]:
import dlem.util as util
from dlem import load_reader, load_model

In [3]:
NUMBER_OF_CHANNELS_PER_ROUTE = 3

In [4]:
BATCH_SIZE = 128 
DATA_FOLDER = "../../../loopextrusion_data_creation/.data/training_data_res_1000_patch_size_500"#".data/dlem_training"
TEST_FOLD = 'fold4'
VAL_FOLD = 'fold5'
LEARNING_RATE = 0.01
PATIENCE = 25
NUM_EPOCH = 250
SAVE_FOLDER = "without_ctcf"

In [5]:
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#dev = torch.device("cpu")

In [8]:
data = load_reader("datareader")(DATA_FOLDER, subselection=[1,2,3])

In [9]:
np.unique(data.data_folds)

array(['fold0', 'fold1', 'fold2', 'fold3', 'fold4', 'fold5', 'fold6',
       'fold7'], dtype=object)

In [10]:
data_test = torch.utils.data.Subset(data, np.where(data.data_folds == TEST_FOLD)[0])
data_val = torch.utils.data.Subset(data, np.where(data.data_folds == VAL_FOLD)[0])
data_train = torch.utils.data.Subset(data, np.where(np.logical_and(data.data_folds != VAL_FOLD,
                                                                   data.data_folds != TEST_FOLD))[0])

In [11]:
dataloader_test = torch.utils.data.DataLoader(data_test, batch_size = BATCH_SIZE, shuffle=True)
dataloader_val = torch.utils.data.DataLoader(data_val, batch_size = BATCH_SIZE, shuffle=True)
dataloader_train = torch.utils.data.DataLoader(data_train, batch_size = BATCH_SIZE, shuffle=True)

In [12]:
index_diagonal = util.diag_index_for_mat(data.patch_dim, data.start_diag, data.stop_diag)

In [13]:
model = load_model("encodetocontact_forked")(data.patch_dim,
                                             data.feature_dim,
                                             data.start_diag,
                                             data.stop_diag,
                                             channel_per_route=NUMBER_OF_CHANNELS_PER_ROUTE).to(dev)

In [14]:
model

DLEM(
  (convs): ModuleList(
    (0): Sequential(
      (0): Conv1d(3, 12, kernel_size=(3,), stride=(1,))
      (1): ReLU()
    )
    (1): Sequential(
      (0): Conv1d(9, 9, kernel_size=(3,), stride=(1,))
      (1): ReLU()
    )
    (2): Sequential(
      (0): Conv1d(6, 6, kernel_size=(3,), stride=(1,))
      (1): ReLU()
    )
    (3): Sequential(
      (0): Conv1d(3, 3, kernel_size=(3,), stride=(1,))
      (1): ReLU()
    )
  )
  (trans_convs): ModuleList(
    (0-3): 4 x Sequential(
      (0): ConvTranspose1d(3, 3, kernel_size=(3,), stride=(1,))
      (1): ReLU()
    )
  )
  (mixer): Sequential(
    (0): Conv1d(12, 2, kernel_size=(1,), stride=(1,))
    (1): Sigmoid()
  )
)

In [15]:
if not os.path.exists(os.path.join(DATA_FOLDER, SAVE_FOLDER)):
    os.mkdir(os.path.join(DATA_FOLDER, SAVE_FOLDER))

In [16]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=PATIENCE, mode='max')
loss = torch.nn.MSELoss(reduction='mean')

In [17]:
diag_init = torch.from_numpy(np.ones((BATCH_SIZE, data.patch_dim), dtype=np.float32) * data.patch_dim)

In [18]:
lr_threshold = 1.5e-6

In [19]:
best_loss = torch.inf
best_corr = -1
mean_loss_traj_train = []
mean_corr_traj_val = []
model = model.to(dev)
diag_init = diag_init.to(dev)
for e in range(NUM_EPOCH):
    training_loss = []
    validation_corr = []
    model.train()
    for diagonals, tracks in dataloader_train:
        optimizer.zero_grad()
        tracks = tracks.to(dev)
        diagonals = diagonals.to(dev)
        total_loss = 0
        for diag_i in range(data.start_diag, data.stop_diag-1):
            next_diag = model(tracks, torch.exp(diagonals[:, index_diagonal(diag_i)]), diag_i)
            total_loss += loss(next_diag, diagonals[:, index_diagonal(diag_i+1)])
        total_loss.backward()
        optimizer.step()
        training_loss.append(total_loss.detach().cpu().numpy())

    mean_total_loss = np.mean(training_loss)
    mean_loss_traj_train.append(mean_total_loss)

    if mean_total_loss < best_loss:
        best_loss = mean_total_loss
        #best_loss_model = copy.deepcopy(model)
        torch.save(model.state_dict(),
                   os.path.join(DATA_FOLDER, SAVE_FOLDER, "best_loss.pt"))

    with torch.no_grad():
        model.eval()

        for diagonals, tracks in dataloader_val:
            tracks = tracks.to(dev)
            diagonals = diagonals.to(dev)
            out = model.contact_map_prediction(tracks, diag_init[:tracks.shape[0]])
            validation_corr.append(util.mat_corr(diagonals, out).detach().cpu().numpy())

        mean_corr = np.mean(validation_corr)
        mean_corr_traj_val.append(mean_corr)

    if mean_corr > best_corr:
        best_corr = mean_corr
        #best_corr_model = copy.deepcopy(model)
        torch.save(model.state_dict(),
                   os.path.join(DATA_FOLDER, SAVE_FOLDER, "best_correlation.pt"))

    scheduler.step(mean_corr)
    if scheduler.get_last_lr()[-1] < lr_threshold:
        break

    print(f'{int((e+1)/NUM_EPOCH*100):3}/100: '
            f'correlation = {mean_corr:.3f}, '
            f'loss = {mean_total_loss:.3f}',
            flush=True, end='\r')

In [ ]:
scheduler.get_last_lr()

In [ ]:
plt.plot(mean_loss_traj_train)

In [ ]:
plt.plot(mean_corr_traj_val)